In [2]:
import numpy as np
import ray
from tokenizers import Tokenizer
from functools import partial
from pathlib import Path
from ray.data.impl.compute import ActorPoolStrategy

In [3]:
data_fp = Path('data/train')
str(data_fp.resolve())

'/Users/dashiell/workspace/simplex-score-matching/data/train'

In [ ]:
data_fp = Path('data/train')
dataset = ray.data.read_numpy('data/train').repartition(8).experimental_lazy()

Repartition:   0%|                                                                                                                  | 0/8 [00:16<?, ?it/s]

In [28]:

class TokenToProbsProcessor:

    def __init__(self, rng, concentration: float, vocab_size: int):
        self.rng = rng
        self.concentration = concentration
        self.vocab_size = vocab_size

    def __call__(self, tokens):
        batch, seq_len, = tokens.shape

        def _tokens_to_probs(token_ids):
            x = self.rng.random((seq_len, self.vocab_size), dtype=np.float32) / self.vocab_size
            # At this point E(x.sum()) == 0.5 
            # What we want is for new_val / (x.sum() + new_val) ~ concentration
            # --> new_val == (concentration * x.sum())/(1 - concentration)
            # Then, in the normalized vector, the appropriate token will have ~ concentration weight,
            # and the others will have the rest
            x_sum = x.sum(axis=1)
            conc_val = np.mean((self.concentration * x_sum) / (1 - self.concentration))
            np.put_along_axis(x, token_ids[:, None], conc_val, axis=1)
            return x / x.sum(axis=1)[:, None]
            
        return np.apply_along_axis(_tokens_to_probs, axis=1, arr=tokens)
    
    
t2p = TokenToProbsProcessor(np.random.default_rng(332), 0.85, 8192)

        

In [29]:
pipe = dataset.map_batches(t2p, batch_size=4).iter_rows()

In [30]:
t2p(next(pipe))

Map_Batches:   0%|                                                                                                                | 0/100 [00:33<?, ?it/s]

KeyboardInterrupt: 

2022-06-19 15:14:29,264	ERROR worker.py:94 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::_map_block_nosplit() (pid=15205, ip=127.0.0.1)
  File "/Users/dashiell/miniconda3/envs/ssm/lib/python3.9/site-packages/ray/data/impl/compute.py", line 341, in _map_block_nosplit
    for new_block in fn(block):
  File "/Users/dashiell/miniconda3/envs/ssm/lib/python3.9/site-packages/ray/data/dataset.py", line 355, in transform
    applied = fn(view)
  File "/var/folders/6s/zv4ygprx6jvg4d7jsgbjfc1r0000gn/T/ipykernel_15145/3303511151.py", line 23, in __call__
  File "<__array_function__ internals>", line 180, in apply_along_axis
  File "/Users/dashiell/miniconda3/envs/ssm/lib/python3.9/site-packages/numpy/lib/shape_base.py", line 402, in apply_along_axis
    buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))
  File "/var/folders/6s/zv4ygprx6jvg4d7jsgbjfc1r0000gn/T/ipykernel_15145/3303511151.py", line 21, in _tokens_to_probs
  File "/Users/dashiell/miniconda3/env